In [1]:
import os
import copy
import pickle
import sympy
import symengine as se
import functools
import itertools
import psutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from error_injection import MissingValueError, SamplingError, Injector
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.metrics import mutual_info_score, auc, roc_curve, roc_auc_score, f1_score
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize as scipy_min
from scipy.spatial import ConvexHull
from scipy.optimize import minimize, Bounds, linprog
from sympy import Symbol as sb
from sympy import lambdify
from tqdm.notebook import trange,tqdm
from IPython.display import display,clear_output
from random import choice

import cProfile
import time
import pstats

from dowhy import CausalModel
from dowhy import causal_estimators
import dowhy.datasets

import ginac_module
import yep
import sys
from multiprocessing import Pool, cpu_count
import math
import regex
import string
from functools import partial

np.random.seed(1)

In [2]:
profiler = cProfile.Profile()
sys.setrecursionlimit(10000)

In [3]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
def create_symbol(suffix=''):
    global symbol_id
    symbol_id += 1
    name = f'e{symbol_id}_{suffix}' if suffix else f'e{symbol_id}'
    return sympy.Symbol(name=name)

In [5]:
# helper functions from causal inference not appeared in zono_reg library

symbol_id = -1
scaler_symbols = set([sb(f'k{i}') for i in range(100)])
linearization_dict = dict()
reverse_linearization_dict = dict()


def compute_closed_form(X, y):
    return np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))

# def sample_data(imputed_datasets, uncert_inds=[], seed=42):
#     imp_np = np.array(imputed_datasets)
#     if len(uncert_inds) == 0:
#         uncert_inds = list(itertools.product(range(imp_np.shape[1]),range(imp_np.shape[2])))
#     np.random.seed(seed)
#     choices = np.random.choice(np.arange(imp_np.shape[0]), len(uncert_inds), replace=True)
#     sample_result = imputed_datasets[0].copy()
#     for i, ind in enumerate(uncert_inds):
#         sample_result[ind[0]][ind[1]] = imputed_datasets[choices[i]][ind[0]][ind[1]]
#     return sample_result

def sample_data(uncert_inds, uncertain_attr, X_extended_max, X_extended_min, seed=42):
    np.random.seed(seed)
    sample_result = []
    for u in uncert_inds:
        maximum = X_extended_max[u, uncertain_attr]
        minimum = X_extended_min[u, uncertain_attr]
        sample_result.append(np.random.uniform(minimum, maximum))
    return sample_result

# X_extended_max[:, uncertain_attr]


def linearization(expr_ls):
    # processed_expr_ls = [0 for _ in range(len(expr_ls))]
    processed_expr_ls = []
    print(len(expr_ls))
    for expr_id, expr in enumerate(expr_ls):
        print("Linearization", expr_id)
        # Do not support monomial expr currently, e.g., expr = 1.5*e1. 
        # At lease two monomials in expr, e.g., expr = 1.5*e1 + 2.
        if not(expr.free_symbols):
            processed_expr_ls[expr_id] += expr
            continue
        expr = expr.expand()
        processed_expr_ls.append(expr)
        # for arg in expr.args:
        #     print(arg)
        #     if not(arg.free_symbols):
        #         processed_expr_ls[expr_id] += arg
        #         continue
        #     p = arg.as_poly()
        #     monomial_exponents = p.monoms()[0]
            
        #     # only deal with non-linear monomials (order > 2)
        #     if sum(monomial_exponents) <= 1:
        #         processed_expr_ls[expr_id] += arg
        #         continue

        #     monomial = sympy.prod(x**k for x, k in zip(p.gens, monomial_exponents) 
        #                           if not(x in scaler_symbols))
        #     # check global substitution dictionary
        #     if monomial in linearization_dict:
        #         processed_expr_ls[expr_id] += arg.coeff(monomial)*linearization_dict[monomial]
        #     else:
        #         print("not in dict")
        #         found = False
        #         subs_monomial = create_symbol()
        #         for symb in monomial.free_symbols:
        #             if symb in reverse_linearization_dict:
        #                 equivalent_monomial = monomial.subs(symb, reverse_linearization_dict[symb])
        #                 if equivalent_monomial in linearization_dict:
        #                     subs_monomial = linearization_dict[equivalent_monomial]
        #                     found = True
        #                     break
        #         linearization_dict[monomial] = subs_monomial
        #         if not(found):
        #             reverse_linearization_dict[subs_monomial] = monomial
        #         processed_expr_ls[expr_id] += arg.coeff(monomial)*subs_monomial
                
    return processed_expr_ls


def merge_small_components_pca(expr_ls, budget=10):
    if not(isinstance(expr_ls, sympy.Expr)):
        expr_ls = sympy.Matrix(expr_ls)
    if expr_ls.free_symbols:
        center = expr_ls.subs(dict([(symb, 0) for symb in expr_ls.free_symbols]))
    else:
        return expr_ls
    monomials_dict = get_generators(expr_ls)
    generators = np.array([monomials_dict[m] for m in monomials_dict])
    if len(generators) <= budget:
        return expr_ls
    monomials = [m for m in monomials_dict]
    pca = PCA(n_components=len(generators[0]))
    pca.fit(np.concatenate([generators, -generators]))
    transformed_generators = pca.transform(generators)
    transformed_generator_norms = np.linalg.norm(transformed_generators, axis=1, ord=2)
    # from largest to lowest norm
    sorted_indices = transformed_generator_norms.argsort()[::-1].astype(int)
    sorted_transformed_generators = transformed_generators[sorted_indices]
    sorted_monomials = [monomials[idx] for idx in sorted_indices]
    new_transformed_generators = np.concatenate([sorted_transformed_generators[:budget], 
                                                 np.diag(np.sum(np.abs(sorted_transformed_generators[budget:]), 
                                                                axis=0))])
    new_generators = pca.inverse_transform(new_transformed_generators)
    new_monomials = sorted_monomials[:budget] + [create_symbol() for _ in range(len(generators[0]))]
    
    processed_expr_ls = center
    for monomial_id in range(len(new_monomials)):
        processed_expr_ls += sympy.Matrix(new_generators[monomial_id])*new_monomials[monomial_id]
    
    return processed_expr_ls


def get_vertices(affset):
    l = len(affset)
    distinct_symbols = set()
    for expr in affset:
        if not(isinstance(expr, sympy.Expr)):
            assert isinstance(expr, int) or isinstance(expr, float)
        else:
            if distinct_symbols:
                distinct_symbols = distinct_symbols.union(expr.free_symbols)
            else:
                distinct_symbols = expr.free_symbols
    distinct_symbols = list(distinct_symbols)
    # print(distinct_symbols)
    combs = [list(zip(distinct_symbols,list(l))) for l in list(itertools.product([-1, 1], repeat=len(distinct_symbols)))]
    res = set()
    for assignment in combs:
        res.add(tuple([expr.subs(assignment) for expr in affset]))
    return(res)

def count_matrix_terms(matrix):
    total_terms = 0
    
    # Iterate through each element in the matrix
    for i in range(matrix.rows):
        for j in range(matrix.cols):
            element = matrix[i, j]
            
            # Count terms in this element
            if element.is_Add:
                # If the element is an Add expression (has + or - operations)
                total_terms += len(element.as_ordered_terms())
            elif element != 0:
                # If the element is a single term (not zero)
                total_terms += 1
                
    return total_terms

In [6]:
def sympy_to_ginac_format(expr):
    """Convert SymPy expression to GiNaC-parseable string format."""
    # Replace SymPy-specific functions with GiNaC equivalents
    s = str(expr)
    s = s.replace('**', '^')  # Exponentiation
    s = s.replace('sin', 'GiNaC::sin')
    s = s.replace('cos', 'GiNaC::cos')
    # Add more replacements as needed
    return s

def sympy_matrix_to_ginac(matrix):
    """Convert SymPy matrix to GiNaC-parseable list of string format."""
    flattened_list = list(matrix.flat())
    flattened_list = [sympy_to_ginac_format(expr) for expr in flattened_list]
    return flattened_list

def ginac_matrix_to_sympy(matrix, rows, cols, symbols_dict):
    POWER_PATTERN = regex.compile(r'\^')
    matrix_list = []
    math_funcs = {
        'sin': sympy.sin, 
        'cos': sympy.cos, 
        'tan': sympy.tan,
        'exp': sympy.exp, 
        'log': sympy.log, 
        'sqrt': sympy.sqrt
    }
    for i in range(rows):
        for j in range(cols):
            idx = i * cols + j
            print(f"converting entry {idx} back to SymPy")
            expr_str = POWER_PATTERN.sub('**', matrix[idx])
            symengine_expr = se.S(expr_str)
            matrix_list.append(symengine_expr)
            # row.append(eval(expr_str, {"__builtins__": {}}, {**symbols_dict, **math_funcs}))

        # matrix_list.append(row)

    se_matrix = se.DenseMatrix(rows, cols, matrix_list)

    print("converting to sympy matrix")
    return sympy.Matrix(rows, cols, [sympy.sympify(se_matrix[i, j]) 
                                for i in range(rows) 
                                for j in range(cols)])

In [7]:
def parse_ginac_tokens(ginac_expr_str, sympy_symbols):
    """
    A string-based approach to convert GiNaC expressions to SymPy
    without any recursion, with support for scientific notation
    
    Parameters:
    -----------
    ginac_expr_str : str
        String representation of a GiNaC expression
    sympy_symbols : dict
        Dictionary mapping symbol names to SymPy symbols
        
    Returns:
    --------
    sympy.Expr
        The equivalent SymPy expression
    """
    # Start timing for performance monitoring
    start_time = time.time()
    
    # Create a case-sensitive symbol map to ensure accurate matching
    symbol_namespace = {}
    for i, (name, symbol) in enumerate(sympy_symbols.items()):
        symbol_namespace[f"sym_{i}"] = symbol
        
    # Add special values to the namespace
    symbol_namespace["nan"] = sympy.nan  # Handle 'nan' as sympy.nan
    
    # Create a reverse mapping for symbol replacement
    symbol_replacements = {}
    for i, name in enumerate(sympy_symbols.keys()):
        # Use word boundaries to ensure we only replace whole symbols
        symbol_replacements[r'\b' + regex.escape(name) + r'\b'] = f"sym_{i}"
    
    # Add special case for 'nan' (must be a whole word)
    # This should NOT be replaced if it's part of a scientific notation
    symbol_replacements[r'\bnan\b'] = "nan"  # Keep 'nan' as is since it's in namespace
    
    # Sort replacements by length (longest first) to avoid substring issues
    sorted_replacements = sorted(
        symbol_replacements.items(), 
        key=lambda x: len(x[0]), 
        reverse=True
    )
    
    print(f"Prepared symbol mappings in {time.time() - start_time:.2f} seconds")
    
    # Split the expression into manageable chunks at + and - operators
    chunks = []
    current_chunk = ""
    i = 0
    
    # Use a faster single-pass approach for the initial split
    while i < len(ginac_expr_str):
        char = ginac_expr_str[i]
        
        # Check if this is scientific notation
        is_sci_notation = False
        if char in ['+', '-'] and i > 0:
            # Look back to see if this is part of scientific notation
            if ginac_expr_str[i-1].lower() == 'e' and i >= 2 and ginac_expr_str[i-2].isdigit():
                is_sci_notation = True
        
        # Handle plus/minus at top level (not in scientific notation)
        if char in ['+', '-'] and not is_sci_notation:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = char if char == '-' else ''
        else:
            current_chunk += char
        
        i += 1
    
    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk)
    
    print(f"Split into {len(chunks)} chunks in {time.time() - start_time:.2f} seconds")
    
    # Process chunks in parallel
    chunk_size = 1000  # Adjust based on your system
    batches = [chunks[i:i + chunk_size] for i in range(0, len(chunks), chunk_size)]
    
    results = []
    for batch_idx, batch in enumerate(batches):
        batch_start = time.time()
        
        # Process this batch in parallel
        n_processes = min(cpu_count(), len(batch))
        if n_processes > 1:
            with Pool(processes=n_processes) as pool:
                process_func = partial(
                    process_term, 
                    sorted_replacements=sorted_replacements,
                    symbol_namespace=symbol_namespace
                )
                batch_results = pool.map(process_func, batch)
        else:
            # For small batches, avoid Pool overhead
            batch_results = [
                process_term(
                    term, 
                    sorted_replacements=sorted_replacements,
                    symbol_namespace=symbol_namespace
                ) 
                for term in batch
            ]
        
        # Combine results from this batch
        batch_result = sum(batch_results)
        results.append(batch_result)
        
        print(f"Processed batch {batch_idx+1}/{len(batches)} in {time.time() - batch_start:.2f} seconds")
    
    # Combine all batch results
    final_result = sum(results)
    print(f"Total processing time: {time.time() - start_time:.2f} seconds")
    
    return final_result

def process_term(term, sorted_replacements, symbol_namespace):
    """
    Process a single term from the expression.
    Uses pre-computed symbol replacements for consistent handling.
    """
    # Handle scientific notation and exponentiation
    term = term.replace("^", "**")
    
    # Process scientific notation in parentheses first
    processed_term = ""
    i = 0
    
    while i < len(term):
        # Scientific notation in parentheses (e.g., (6.741051087966850411E-11))
        if i < len(term) - 1 and term[i] == '(' and term[i+1].isdigit():
            # Find the end of the parenthesized scientific notation
            end_paren = term.find(')', i)
            if end_paren != -1:
                sci_notation = term[i+1:end_paren]
                # Convert to SymPy Float for precision
                processed_term += f"sympy.Float('{sci_notation}')"
                i = end_paren + 1
                continue
        
        # Regular character
        processed_term += term[i]
        i += 1
    
    # Handle special constants before symbol replacements
    # Replace 'nan' that is not part of a scientific notation with sympy.nan
    processed_term = regex.sub(r'\bnan\b', 'sympy.nan', processed_term)
    
    # Now apply all symbol replacements in order (longest first)
    for pattern, replacement in sorted_replacements:
        processed_term = regex.sub(pattern, replacement, processed_term)
    
    # Create namespace for evaluation
    namespace = {
        "sympy": sympy,
        **symbol_namespace
    }
    
    try:
        # Compile to bytecode first
        compiled_expr = compile(processed_term, '<string>', 'eval')
        return eval(compiled_expr, {"__builtins__": {}}, namespace)
    except Exception as e:
        # More detailed error reporting
        print(f"Error processing term (first 50 chars): '{term[:50]}...': {e}")
        print(f"Processed term was: '{processed_term[:100]}...'")
        # Return zero for this term but continue processing
        return sympy.S.Zero

# Helper function to check for other special mathematical constants
def check_for_special_constants(expr_str):
    """
    Identify special mathematical constants in the expression
    """
    constants = {
        'nan': 'Not a number',
        'inf': 'Infinity',
        'pi': 'Pi constant',
        'e': 'Euler\'s number',
        # Add other constants if needed
    }
    
    found = {}
    for const in constants:
        pattern = r'\b' + const + r'\b'
        if regex.search(pattern, expr_str):
            found[const] = constants[const]
    
    if found:
        print("Found special constants:")
        for const, desc in found.items():
            print(f"  - {const}: {desc}")
    
    return found

# Optional: Add this debugging function to help identify symbol issues
def debug_symbol_matching(expr_str, sympy_symbols):
    """Debug which symbols are found and which are missing"""
    # Find all potential symbol patterns
    symbol_pattern = r'\be[a-zA-Z]?[0-9]+\b'
    found_symbols = set(regex.findall(symbol_pattern, expr_str))
    
    # Check which ones are in the provided dictionary
    missing = [s for s in found_symbols if s not in sympy_symbols]
    present = [s for s in found_symbols if s in sympy_symbols]
    
    print(f"Total potential symbols found: {len(found_symbols)}")
    print(f"Symbols in dictionary: {len(present)}")
    print(f"Symbols not in dictionary: {len(missing)}")
    
    if missing:
        print(f"First 10 missing symbols: {missing[:10]}")
    
    # Check for case-sensitivity issues
    sympy_keys_lower = [k.lower() for k in sympy_symbols.keys()]
    case_issues = [s for s in missing if s.lower() in sympy_keys_lower]
    
    if case_issues:
        print(f"Possible case-sensitivity issues: {case_issues[:10]}")

# Additional utility function for memory tracking (optional)
def get_memory_usage():
    """Return current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

In [8]:
def zorro(ss, X_train, y_train, X_test, y_test, robustness_radius, uncertain_attr, 
                                 uncertain_num, uncertain_radius=None, uncertain_radius_ratio=None, 
                                 lr=0.03, reg=0, seed=42):
    start_time = time.perf_counter()
    X = copy.deepcopy(X_train)
    y = copy.deepcopy(y_train)
    
    symbol_id = -1
    symbolic_data = X.tolist()
    symbols_in_data = set()
    symbol_to_position = dict()
    symbols_dict = dict()
    XS = X.tolist()
    XR = X.tolist()
    for row in range(len(symbolic_data)):
        for col in range(len(symbolic_data[0])):
            xmin = X_extended_min[row][col]
            xmax = X_extended_max[row][col]
            if xmin != xmax:
                xmean = (xmax + xmin) / 2
                xradius = (xmax - xmin) / 2
                new_symbol = create_symbol()
                symbolic_data[row][col] = xmean + xradius*new_symbol
                XS[row][col] = xradius*new_symbol
                XR[row][col] = xmean
                symbols_in_data.add(new_symbol)
                symbol_to_position[new_symbol] = (row, col)
                symbols_dict[str(new_symbol)] = new_symbol
            else:
                XS[row][col] = 0
                
    XS = sympy.Matrix(XS)
    XR = sympy.Matrix(XR)
    n = XS.shape[0]
    n1 = XS.shape[1]
    yR = sympy.Matrix(y_train)
    yS = yR*0.0
    y_test_orig = y_test.copy()
    y_test = sympy.Matrix(y_test.to_numpy().reshape(-1, 1))
    # X_test = sympy.Matrix(np.append(np.ones((len(X_test), 1)), X_test, axis=1))
    X_test = sympy.Matrix(np.append(np.ones((len(X_test), 1)), ss.transform(X_test), axis=1))

    # for row in range(n):
    #     for col in range(n1):
    #         expr = copy.deepcopy(XR[row, col])
    #         if isinstance(expr, sympy.Expr) and expr.free_symbols:
    #             XR[row, col] = expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))
    #             XS[row, col] = expr - XR[row, col]
    #         else:
    #             XR[row, col] = expr
    #             XS[row, col] = 0

    # for row in range(yR.shape[0]):
    #     expr = copy.deepcopy(yR[row])
    #     if isinstance(expr, sympy.Expr) and expr.free_symbols:
    #         yR[row] = expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))
    #         yS[row] = expr - yR[row]
    #     else:
    #         yR[row] = expr
    #         yS[row] = 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Data loading execution time: {execution_time} seconds")

    start_time = time.perf_counter()
    common_inv = (XR.T*XR + reg*n*np.identity(X.shape[1])).inv()
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Common inverse matrix computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    V, sigma, VT = np.linalg.svd(np.array((XR.T*XR).tolist()).astype(float))
    V = sympy.Matrix(V)
    VT = sympy.Matrix(VT)
    print("VT: ", VT)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"SVD computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    wR = common_inv*XR.T*yR
    wS_data = common_inv*((XS.T*XR + XR.T*XS)*wR - XS.T*yR - XR.T*yS)
    wS_non_data = 0.0*VT.row(0).T
    for i in range(X.shape[1]):
        wS_non_data = wS_non_data + sb(f'k{i}')*sb(f'ep{i}')*VT.row(i).T
        symbols_dict[f"ep{i}"] = sb(f'ep{i}')
        symbols_dict[f"k{i}"] = sb(f'k{i}')
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Abstract weights computation time: {execution_time} seconds")
    # print(symbols_dict)

    start_time = time.perf_counter()
    eigenvalues = 1 - 2*lr*reg - 2*lr*sigma/n
    for eigenvalue in eigenvalues:
        assert eigenvalue <= 1
        assert eigenvalue >= 0
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Eigenvalue computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    wS = wS_non_data + wS_data
    w = wS + wR
    w_prime = (-lr*2/n)*((XS.T*XR + XR.T*XS + XS.T*XS)*wS + XS.T*XS*wR - XS.T*yS).expand()
    # w_prime = (XS.T*XR + XR.T*XS + XS.T*XS)*wS + XS.T*XS*wR - XS.T*yS
    # print("Before expansion:", w_prime.rows, w_prime.cols)
    # w_prime = ginac_matrix_to_sympy(ginac_module.expand_matrix(sympy_matrix_to_ginac(w_prime), w_prime.rows, w_prime.cols), w_prime.rows, w_prime.cols, symbols_dict)
    # print("After expansion:", w_prime.rows, w_prime.cols)
    # w_prime = (-lr*2/n)*w_prime
    # print("w_prime final:", w_prime.rows, w_prime.cols, "\n")
    
    coeff_cont = (-lr*2/n)*((XS.T*XR + XR.T*XS + XS.T*XS)*wS_non_data).expand()
    # coeff_cont = (XS.T*XR + XR.T*XS + XS.T*XS)*wS_non_data
    # print("Before expansion:", coeff_cont.rows, coeff_cont.cols)
    # coeff_cont = ginac_matrix_to_sympy(ginac_module.expand_matrix(sympy_matrix_to_ginac(coeff_cont), coeff_cont.rows, coeff_cont.cols), coeff_cont.rows, coeff_cont.cols, symbols_dict)
    # print("After expansion:", coeff_cont.rows, coeff_cont.cols)
    # coeff_cont = (-lr*2/n)*coeff_cont
    # print("coeff_cont final:", coeff_cont.rows, coeff_cont.cols)

    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"W# (abstract gradient descent?) computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    w_prime_projected = VT*w_prime
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Projection computation time: {execution_time} seconds")

    start_time = time.perf_counter()
    eqs = []
    for d in tqdm(range(X.shape[1]), desc='Equation'):
        eq1 = (1-abs(eigenvalues[d]))*sb(f'k{d}')
        eq2 = 0
        coef_dict = dict()
        coef_dict['const'] = 0
        for i in range(X.shape[1]):
            coef_dict[sb(f'k{i}')] = 0
        for arg in w_prime_projected[d].args:
            contain_k = False
            for i in range(X.shape[1]):
                symb_k = sb(f'k{i}')
                if symb_k in arg.free_symbols:
                    coef_dict[symb_k] = coef_dict[symb_k] + abs(arg.args[0])
                    contain_k = True
                    break
            if not(contain_k):
                coef_dict['const'] = coef_dict['const'] + abs(arg.args[0])
        eq2 = coef_dict['const']
        for i in range(X.shape[1]):
            eq2 = eq2 + sb(f'k{i}')*coef_dict[sb(f'k{i}')]
        eqs.append(sympy.Eq(eq1, eq2))
        
    result = sympy.solve(eqs, [sb(f'k{i}') for i in range(X.shape[1])])
    print(result)
    for ki in result:
        assert result[ki] >= 0
    param = wR + wS.subs(result)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Equation solving time: {execution_time} seconds")
    
    # center_preds = X_test*get_expr_center(param)
    # print('one-poss-world:', [((center_preds - y_test).T*(center_preds - y_test)/n)[0]])
    # start_time = time.perf_counter()
    # test_preds = X_test*param
    # end_time = time.perf_counter()
    # execution_time = end_time - start_time
    # print(f"Prediction time: {execution_time} seconds")
    # print(test_preds[0])
    # robustness_ls = []
#     se_min_ls = []
#     se_max_ls = []
    # acc_count_low = 0
    # acc_count_high = 0
    # for pred_id in tqdm(range(len(test_preds)), desc='Testing'):
    #     pred = test_preds[pred_id]
    #     label_test = y_test[pred_id]
    #     pred_range_radius = get_expr_range_radius(pred)
    #     if pred_range_radius <= robustness_radius:
    #         robustness_ls.append(1)
    #     else:
    #         robustness_ls.append(0)
    #     pred_center = get_expr_center(pred)
    #     pred_low = pred_center - pred_range_radius
    #     pred_hi = pred_center + pred_range_radius
    #     # get accuracy lower and upper bound
    #     if pred_hi < 0.5 and label_test == 0:
    #         acc_count_low += 1
    #         acc_count_high += 1
    #     elif pred_low > 0.5 and label_test == 1:
    #         acc_count_low += 1
    #         acc_count_high += 1
    #     elif pred_low < 0.5 and pred_hi > 0.5:
    #         acc_count_high += 1
#         rewrite_pred = pred_center+pred_range_radius*sb('err_pred')
#         se_min, se_max = min_max_sympy_expression((rewrite_pred-label_test)**2)
#         se_min_ls.append(se_min)
#         se_max_ls.append(se_max)
    # preds_diff = (test_preds - y_test).T*(test_preds - y_test)/n
    start_time = time.perf_counter()
    ginac_param = sympy_to_ginac_format(param)
    X_test_list = list(np.array(X_test, dtype=float))
    X_test_list = [list(l) for l in X_test_list]
    y_test_list = [float(l[0]) for l in y_test_orig.to_numpy()]
    ginac_param_list = [sympy_to_ginac_format(s) for s in param]
    abstract_loss_str = ginac_module.abstract_loss(X_test_list, y_test_list, ginac_param_list)

    POWER_PATTERN = regex.compile(r'\^')
    math_funcs = {
        'sin': sympy.sin, 
        'cos': sympy.cos, 
        'tan': sympy.tan,
        'exp': sympy.exp, 
        'log': sympy.log, 
        'sqrt': sympy.sqrt
    }
    expr_str = POWER_PATTERN.sub('**', abstract_loss_str[0])
    
    # preds_diff_full = eval(expr_str, {"__builtins__": {}}, {**symbols_dict, **math_funcs})
    # compiled_expr = compile(expr_str, '<string>', 'eval')
    # preds_diff_full = eval(compiled_expr, symbols_dict)
    preds_diff_full = parse_ginac_tokens(abstract_loss_str[0], symbols_dict)
    end_time = time.perf_counter()
    execution_time = end_time - start_time
    print(f"Loss computation and expansion time: {execution_time} seconds")
    
    # start_time = time.perf_counter()
    # n = X_test.shape[0]
    # preds_diff = ((test_preds - y_test).T*(test_preds - y_test)/n)[0]
    # end_time = time.perf_counter()
    # execution_time = end_time - start_time
    # print(f"Loss computation time: {execution_time} seconds")

    # start_time = time.perf_counter()
    # preds_diff_full = linearization([preds_diff])[0]
    # preds_diff_center = get_expr_center(preds_diff)
    # preds_diff_radius = get_expr_range_radius(preds_diff)
#     print(preds_diff_center)
#     print(preds_diff_radius)
    # print("Robustness Ratio: " + str(np.mean(robustness_ls)))
    
#     print(param)
    # end_time = time.perf_counter()
    # execution_time = end_time - start_time
    # print(f"Loss linearization time: {execution_time} seconds")
    return result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff_full, X_test

In [9]:
def get_expr_range_radius(expr):
    expr_range_radius = 0
    for arg in expr.args:
        if arg.free_symbols:
            expr_range_radius += abs(arg.args[0])
    return expr_range_radius

def get_expr_center(expr):
    return expr.subs(dict([(symb, 0) for symb in expr.free_symbols]))

In [10]:
X_train = pd.read_csv('dataset/Insurance_injector_lr/real/X_train_clean.csv')
X_train_dirty = pd.read_csv('dataset/Insurance_injector_lr/real/X_train_dirty.csv')

In [11]:
injected_size = 127

In [12]:
y_train = pd.read_csv('dataset/Insurance_injector_lr/real/y_train.csv')
y_train.shape[0]

In [13]:
y_train = pd.DataFrame(y_train, columns=['charges'])
y_train

,charges
0,0
1,1
2,1
3,1
4,1
...,...
797,1
798,0
799,1
800,0


In [14]:
df = X_train.merge(y_train, left_index=True, right_index=True)
def compute_closed_form(X, y):
    return np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y))

In [15]:
dirty_df = X_train_dirty.merge(y_train, left_index=True, right_index=True)

In [16]:
all_cols = ['age', 'bmi', 'children']
all_cols_idx = [df.columns.to_list().index(c) for c in all_cols]

# num of errors injected
# mv_num = 30

# which col to inject missing val
treatment_idx = all_cols.index('bmi')
uncertain_attr = all_cols.index('children')
# uncertain_attr = [all_cols.index('displacement'), all_cols.index('weight')]

# MNAR
# def non_random_pattern(data_X, data_y):
#     binary_indicators = []
#     for i in range(data_X.shape[0]):
#         if (data_X.iloc[i, treatment_idx] > 25) and (data_X.iloc[i, uncertain_attr] > 0):
#             binary_indicators.append(1)
#         else:
#             binary_indicators.append(0)
#     return np.array(binary_indicators)

# sample_pattern_len = np.sum(non_random_pattern(df.drop('cylinders', axis=1).copy(), df.cylinders))
# mv_ratio = mv_num/sample_pattern_len
# print(mv_ratio)
# mv_err = MissingValueError(uncertain_attr, pattern=non_random_pattern, ratio=mv_ratio)
# dirty_df, dirty_y, _, _ = mv_err.inject(df.drop('cylinders', axis=1).copy(), df.cylinders, df.drop('cylinders', axis=1), df.cylinders)

X_extended = np.append(np.ones((len(dirty_df), 1)), 
                       dirty_df.to_numpy().astype(float)[:, all_cols_idx], axis=1)
X_extended_clean = np.append(np.ones((len(df), 1)), 
                             df.to_numpy().astype(float)[:, all_cols_idx], axis=1)

ss = StandardScaler()
X_extended[:, 1:] = ss.fit_transform(X_extended[:, 1:])
X_extended_clean[:, 1:] = ss.transform(X_extended_clean[:, 1:])

param_clean = compute_closed_form(X_extended_clean, y_train)

# first column becomes constant 1's
treatment_idx += 1
uncertain_attr += 1
# remain_col_idx = [0] + [i+1 for i in remain_col_idx]

In [17]:
imputers = [KNNImputer(n_neighbors=3), KNNImputer(n_neighbors=5), KNNImputer(n_neighbors=10), IterativeImputer(random_state=42)]
num_attrs = X_extended.shape[1]
X_nan = X_extended.copy()
imputed_cols = [X_extended_clean[:, uncertain_attr]]
# imputed_other = [X_extended_clean[:, other_uncertain]]
imputed_datasets = [X_extended_clean]
for imp in imputers:
    imputed_dataset = imp.fit_transform(X_nan)
    imputed_datasets.append(imputed_dataset)
    imputed_cols.append(imputed_dataset[:, uncertain_attr])
    # imputed_other.append(imputed_dataset[:, other_uncertain])

X_extended_max = X_extended.copy()
X_extended_max[:, uncertain_attr] = np.max(imputed_cols, axis=0)
# X_extended_max[:, other_uncertain] = np.max(imputed_other, axis=0)
# X_extended_max = X_extended_max[:, remain_col_idx]

X_extended_min = X_extended.copy()
X_extended_min[:, uncertain_attr] = np.min(imputed_cols, axis=0)
# X_extended_min[:, other_uncertain] = np.min(imputed_other, axis=0)
# X_extended_min = X_extended_min[:, remain_col_idx]

# X_extended = X_extended[:, remain_col_idx]

In [18]:
X_test = pd.read_csv('dataset/Insurance_injector_lr/real/X_test.csv')
y_test = pd.read_csv('dataset/Insurance_injector_lr/real/y_test.csv')
dirty_y = y_train

In [19]:
X_val = pd.read_csv('dataset/Insurance_injector_lr/real/X_val.csv')
y_val = pd.read_csv('dataset/Insurance_injector_lr/real/y_val.csv')

In [20]:
df_diff = pd.concat([X_train, X_train_dirty])
df_diff = df_diff.drop_duplicates(keep=False)

In [21]:
injected_indices = []
for i in range(len(dirty_df)):
    if pd.isna(dirty_df.iloc[i]['children']):
        injected_indices.append(i)

injected_indices = np.array(injected_indices)
injected_indices

array([  1,   2,   9,  13,  20,  22,  23,  29,  30,  48,  52,  57,  68,
        71,  73,  91,  93,  96, 104, 105, 107, 108, 118, 128, 129, 130,
       134, 143, 151, 152, 155, 156, 169, 194, 199, 204, 225, 228, 236,
       237, 238, 241, 247, 254, 274, 277, 306, 315, 318, 320, 339, 346,
       352, 354, 361, 365, 366, 367, 368, 375, 378, 383, 384, 391, 393,
       407, 411, 422, 423, 427, 429, 434, 437, 445, 447, 456, 457, 471,
       493, 496, 506, 519, 520, 529, 531, 536, 540, 543, 562, 568, 581,
       585, 587, 588, 592, 598, 615, 632, 636, 638, 656, 657, 663, 672,
       676, 679, 684, 696, 698, 710, 713, 714, 718, 721, 737, 738, 743,
       754, 756, 762, 764, 768, 775, 784, 790, 797, 799])

In [22]:
len(injected_indices)

In [23]:
X_test_extended = np.append(np.ones((len(X_test), 1)), 
                       X_test.to_numpy().astype(float)[:, all_cols_idx], axis=1)
X_test_extended[:, 1:] = ss.transform(X_test_extended[:, 1:])

In [24]:
X_test_list = [list(l) for l in X_test_extended]
y_test_list = [float(l[0]) for l in y_test.to_numpy()]

In [25]:
X_max_list = [list(l) for l in X_extended_max]
X_min_list = [list(l) for l in X_extended_min]

In [26]:
X_train_list = [list(l) for l in X_extended]
y_train_list = [float(l) for l in dirty_y.to_numpy()]

In [27]:
param_list = ginac_module.zorro(X_train_list, y_train_list, X_max_list, X_min_list, 0.01, 0.0)

         802 -8.27014e-07 -1.79652e-06      72.9245
-8.27014e-07          802      82.4512     -15.1908
-1.79652e-06      82.4512          802     -18.0479
     72.9245     -15.1908     -18.0479      729.642
VT -0.186537   0.674334   0.678107  -0.225054
  0.834628    0.20002   0.189039   0.477128
-0.0308626   0.709846  -0.702816  0.0348646
 -0.517346  0.0372023     0.1024   0.848813
W# (abstract gradient descent?) computation time: 418 ms
Projection computation time: 142 ms

Equations:
Equation 0: (0.022194071908910181984)*k0==2.6257738132115946182E-4+(0.0012255736577262612377)*k0+(5.290504460406367584E-4)*k2+(0.0015030183776907821416)*k1+(0.0025778711037544978033)*k3
Equation 1: (0.021039611536784928519)*k1==6.466784723257086926E-4+(0.0015030183776907821407)*k0+(0.0011620313970699118911)*k2+(0.004263465818100308403)*k1+(0.0044895502193954458527)*k3
Equation 2: (0.017945618593692169089)*k2==1.5880358375751634047E-4+(5.290504460406367586E-4)*k0+(1.5860832679283843644E-4)*k2+(0.001162031

In [23]:
result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff_full, X_test_matrix = zorro(ss, X_extended, dirty_y, X_val, y_val, 0.05, uncertain_attr, 
                             injected_size, uncertain_radius=None, uncertain_radius_ratio=None, lr=0.01, reg=0, seed=42)

Data loading execution time: 0.10425499800112448 seconds
Common inverse matrix computation time: 0.08005503499953193 seconds
SVD computation time: 0.05963437500031432 seconds
Abstract weights computation time: 1.1960066510000615 seconds
{'e0': e0, 'e1': e1, 'e2': e2, 'e3': e3, 'e4': e4, 'e5': e5, 'e6': e6, 'e7': e7, 'e8': e8, 'e9': e9, 'e10': e10, 'e11': e11, 'e12': e12, 'e13': e13, 'e14': e14, 'e15': e15, 'e16': e16, 'e17': e17, 'e18': e18, 'e19': e19, 'e20': e20, 'e21': e21, 'e22': e22, 'e23': e23, 'e24': e24, 'e25': e25, 'e26': e26, 'e27': e27, 'e28': e28, 'e29': e29, 'e30': e30, 'e31': e31, 'e32': e32, 'e33': e33, 'e34': e34, 'e35': e35, 'e36': e36, 'e37': e37, 'e38': e38, 'e39': e39, 'e40': e40, 'e41': e41, 'e42': e42, 'e43': e43, 'e44': e44, 'e45': e45, 'e46': e46, 'e47': e47, 'e48': e48, 'e49': e49, 'e50': e50, 'e51': e51, 'e52': e52, 'e53': e53, 'e54': e54, 'e55': e55, 'e56': e56, 'e57': e57, 'e58': e58, 'e59': e59, 'e60': e60, 'e61': e61, 'e62': e62, 'e63': e63, 'e64': e64, 'e

Equation:   0%|          | 0/4 [00:00<?, ?it/s]

{k0: 0.0266298794545128, k1: 0.0622455095121689, k2: 0.0219989802986302, k3: 0.0739433927527262}
Equation solving time: 1.8751018620005198 seconds
X: 268*4
y: 268*1
param: 4*1
Time for multiplication: 26 ms
Time for difference: 2 ms
Time for square and expand: 88874 ms
Number of nodes in sum_squared: 552298
Number of terms in sum_squared: 138075
Depth of sum_squared: 3
Current memory usage: 	  878760 kB
Symbol extraction time: 83 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 5914 ms
Time for combining like terms: 6638 ms
Number of nodes in sum_squared after expansion: 69050
Number of terms in sum_squared after expansion: 17263
Depth of sum_squared after expansion: 3
Current memory usage: 	  870984 kB
Loss computation and expansion time: 1517.125285799999 seconds


In [22]:
profiler.enable()
result, param, wS_data, wS_non_data, wR, w_prime, w_prime_projected, preds_diff, preds_diff_full, X_test_matrix = zorro(ss, X_extended, dirty_y, X_val, y_val, 0.05, uncertain_attr, 
                             injected_size, uncertain_radius=None, uncertain_radius_ratio=None, lr=0.01, reg=0, seed=42)

Data loading execution time: 0.25000879999970493 seconds
Common inverse matrix computation time: 0.18765725100001873 seconds
SVD computation time: 0.14930486899993411 seconds
Abstract weights computation time: 3.0665186189999076 seconds
{'e0': e0, 'e1': e1, 'e2': e2, 'e3': e3, 'e4': e4, 'e5': e5, 'e6': e6, 'e7': e7, 'e8': e8, 'e9': e9, 'e10': e10, 'e11': e11, 'e12': e12, 'e13': e13, 'e14': e14, 'e15': e15, 'e16': e16, 'e17': e17, 'e18': e18, 'e19': e19, 'e20': e20, 'e21': e21, 'e22': e22, 'e23': e23, 'e24': e24, 'e25': e25, 'e26': e26, 'e27': e27, 'e28': e28, 'e29': e29, 'e30': e30, 'e31': e31, 'e32': e32, 'e33': e33, 'e34': e34, 'e35': e35, 'e36': e36, 'e37': e37, 'e38': e38, 'e39': e39, 'e40': e40, 'e41': e41, 'e42': e42, 'e43': e43, 'e44': e44, 'e45': e45, 'e46': e46, 'e47': e47, 'e48': e48, 'e49': e49, 'e50': e50, 'e51': e51, 'e52': e52, 'e53': e53, 'e54': e54, 'e55': e55, 'e56': e56, 'e57': e57, 'e58': e58, 'e59': e59, 'e60': e60, 'e61': e61, 'e62': e62, 'e63': e63, 'e64': e64, 'e

Equation:   0%|          | 0/4 [00:00<?, ?it/s]

{k0: 0.0266298794545128, k1: 0.0622455095121689, k2: 0.0219989802986302, k3: 0.0739433927527263}
Equation solving time: 4.865998120000768 seconds



KeyboardInterrupt



In [ ]:
profiler.disable()
stats2 = pstats.Stats(profiler)
stats2.sort_stats('cumulative').print_stats(100)

In [54]:
with open('dataset/Insurance_injector_lr/ins_iter_model.pkl', 'wb') as file:
    pickle.dump(param, file)

In [55]:
with open('dataset/Insurance_injector_lr/X_test_matrix.pkl', 'wb') as file:
    pickle.dump(X_test_matrix, file)

In [56]:
with open('dataset/Insurance_injector_lr/y_test_ins.pkl', 'wb') as file:
    pickle.dump(y_test, file)

In [58]:
with open('dataset/Insurance_injector_lr/ins_iter_full.pkl', 'wb') as file:
    pickle.dump(preds_diff_full, file)

with open('dataset/Insurance_injector_lr/ins_iter.pkl', 'wb') as file:
    pickle.dump(preds_diff, file)

In [28]:
with open('dataset/Insurance_injector_lr/ins_iter_model.pkl', 'rb') as file:
    param = pickle.load(file)

In [29]:
with open('dataset/Insurance_injector_lr/X_test_matrix.pkl', 'rb') as file:
    X_test_matrix = pickle.load(file)

In [30]:
with open('dataset/Insurance_injector_lr/y_test_ins.pkl', 'rb') as file:
    y_test = pickle.load(file)

In [31]:
with open('dataset/Insurance_injector_lr/ins_iter.pkl', 'rb') as file:
    preds_diff = pickle.load(file)

In [32]:
ginac_param = sympy_to_ginac_format(param)
X_test_list = list(np.array(X_test_matrix, dtype=float))
X_test_list = [list(l) for l in X_test_list]
y_test_list = [float(l[0]) for l in y_test.to_numpy()]
ginac_param_list = [sympy_to_ginac_format(s) for s in param]

In [33]:
# failed multiprocessing attempts

def worker_expand(expression_chunk_list):
    try:
        # Directly call the C++ function that handles the whole chunk
        partial_sum = ginac_module.expand_chunk(expression_chunk_list)
        # Check if partial_sum indicates C++ error
        if partial_sum.startswith("__CPP_CHUNK_"):
             print(f"!!! C++ Worker Error reported for chunk starting with '{expression_chunk_list[0]}': {partial_sum}")
             return "__WORKER_ERROR__" # Propagate error signal

        print(f"Worker received partial sum string (length {len(partial_sum)})") # Debug print
        return sympy.sympify(partial_sum)
        print("sympify finished")
    except Exception as e:
        print(f"!!! Error calling C++ chunk processor: {e}")
        return "__WORKER_ERROR__" # Indicate error

In [34]:
terms = preds_diff.as_ordered_terms()
terms = [sympy_to_ginac_format(t) for t in terms]
print(terms[0])

3.3278250767467e-5*(-0.0246391269442471*e0 - 0.0252479203275169*e1 - 0.0101175044290985*e10 - 0.0119520231449748*e100 - 0.0330002014650581*e101 - 0.0126281695103206*e102 - 0.00590621901976043*e103 - 0.032754402438976*e104 - 0.0122210921712473*e105 - 0.0176268659750799*e106 - 0.00911668428689498*e107 - 0.0119364000856258*e108 - 0.0231181414459843*e109 - 0.0205900411493115*e11 - 0.0120369498059076*e110 - 0.0144250576318271*e111 - 0.0218803478876332*e112 - 0.00691707063956159*e113 - 0.00611494610828488*e114 - 0.00572042867542511*e115 - 0.0030606435183421*e116 - 0.0210286598848253*e117 - 0.0219377581488873*e118 - 0.00765807939078065*e119 - 0.0137723884571582*e12 - 0.0341137042026003*e120 - 0.00750103405267395*e121 - 0.0061922201751392*e122 - 0.0354053503580802*e123 - 0.0114935404063533*e124 - 0.00733776229255484*e125 - 0.0222969875454784*e126 - 0.00855931998718758*e13 - 0.00754000987666508*e14 - 0.00473676538594785*e15 - 0.00742123929777615*e16 - 0.00860291724890884*e17 - 0.010589572258692

In [35]:
profile_output = 'cpp_profiling_ins.prof'

In [36]:
profiler.clear()
profiler.enable()
start_time = time.perf_counter()
print(f"Starting C++ profiler, output will be in {profile_output}")
yep.start(profile_output)
result_strings = ginac_module.abstract_loss(X_test_list, y_test_list, ginac_param_list)
yep.stop()
print("C++ Profiling completed")
end_time = time.perf_counter()
profiler.disable()
execution_time = end_time - start_time
print(f"GiNaC time: {execution_time} seconds")

Starting C++ profiler, output will be in cpp_profiling_ins.prof
X: 268*4
y: 268*1
param: 4*1
Time for multiplication: 29 ms
Time for difference: 3 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Symbol extraction time: 0 ms
Symbol priority generation time: 0 ms
combine_like_terms_robust() time: 0 ms
Sy

PROFILE: interrupts/evictions/bytes = 65/0/43656


In [37]:
stats2 = pstats.Stats(profiler)
stats2.sort_stats('cumulative').print_stats(100)

         232 function calls in 0.869 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        8    0.000    0.000    0.866    0.108 /home/hky/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        8    0.000    0.000    0.866    0.108 {built-in method builtins.exec}
        1    0.659    0.659    0.659    0.659 {built-in method ginac_module.abstract_loss}
        1    0.202    0.202    0.202    0.202 /home/hky/.local/lib/python3.10/site-packages/yep.py:56(stop)
        1    0.005    0.005    0.005    0.005 /home/hky/.local/lib/python3.10/site-packages/yep.py:32(start)
        8    0.000    0.000    0.003    0.000 /usr/lib/python3.10/codeop.py:117(__call__)
        8    0.003    0.000    0.003    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 /home/hky/.local/lib/python3

In [38]:
print("\nTo analyze the C++ profiler results, run these commands in a terminal:")
print(f"google-pprof --gif {sys.executable} {profile_output} > prof_ins.gif")


To analyze the C++ profiler results, run these commands in a terminal:
google-pprof --gif /usr/bin/python3 cpp_profiling_ins.prof > prof_ins.gif


In [39]:
start_time = time.perf_counter()
result_sympy = []
    
for expr_str in result_strings:
    # Parse the GiNaC output to SymPy
    # This would need a proper parser for real use
    expr = sympy.sympify(expr_str)
    # expr = sympy.expand(expr)
    # expr = sympy.collect(expr, list(symbols))
        
    result_sympy.append(expr)

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"SymPy conversion time: {execution_time} seconds")

print("Result from GiNaC:")
for expr in result_sympy:
    print(expr)

SymPy conversion time: 0.3135848679958144 seconds
Result from GiNaC:
1.5231e-6*e0**2 + 0.000236215*e0 + 1.87099e-6*e1**2 + 0.000177778*e1 + 2.60456e-7*e10**2 + 8.95777e-5*e10 + 5.24194e-7*e100**2 + 4.73107e-5*e100 + 2.78929e-6*e101**2 + 0.000301219*e101 + 5.84805e-7*e102**2 + 5.67344e-5*e102 + 1.1023e-7*e103**2 + 3.26083e-5*e103 + 2.56863e-6*e104**2 + 0.000322736*e104 + 5.16966e-7*e105**2 + 5.37992e-5*e105 + 8.36635e-7*e106**2 + 0.000146559*e106 + 2.1817e-7*e107**2 + 7.87625e-5*e107 + 3.5515e-7*e108**2 + 0.00011504*e108 + 1.38708e-6*e109**2 + 0.000203024*e109 + 1.01428e-6*e11**2 + 0.000211828*e11 + 5.31714e-7*e110**2 + 5.06827e-5*e110 + 6.47701e-7*e111**2 + 8.47513e-5*e111 + 1.11511e-6*e112**2 + 0.000222373*e112 + 1.62014e-7*e113**2 + 3.00569e-5*e113 + 1.44842e-7*e114**2 + 1.48845e-5*e114 + 1.49023e-7*e115**2 + 3.69561e-6*e115 + 8.18635e-8*e116**2 - 2.96036e-5*e116 + 1.35508e-6*e117**2 + 0.000136435*e117 + 1.11915e-6*e118**2 + 0.000223345*e118 + 3.28225e-7*e119**2 - 1.76114e-5*e119 + 6